# Запуск на устройствах пользователей

Чтобы запустить стенд с TfJS , перейдите в директорию `tensorflowjs` и запустите

```bash
docker-compose up
```

После этого можно переходить по адресам
* http://localhost:9090
* http://localhost:9091

## PyTorch. DistributedDataParallel 

Это класс, скрывающий под капотом детали параллельного обучениия в PyTorch, применяется для обучения на кластере из нескольких компьютеров с несколькими GPU

Внутри осуществляет разбивку данных по обработчикам, на backward шаге градиенты усредняются с использованием allreduce.

Существует также `DataParallel` класс, но он работает в рамках одного процесса, используя потоки, тем самым он не рекомендуется к применению в силу ограничений GIL, `DistributedDataParallel` показывает себя лучше даже при обучении на одном компьютере.

In [1]:
%%writefile launch_ddp_demo.py

import os
import sys
import tempfile
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

from torch.nn.parallel import DistributedDataParallel as DDP


def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'

    # инициализация группы процессов
    dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = nn.Linear(10, 10)
        self.relu = nn.ReLU()
        self.net2 = nn.Linear(10, 5)

    def forward(self, x):
        return self.net2(self.relu(self.net1(x)))


def demo_basic(rank, world_size):
    print(f"Базовый пример использованиия DDP с рангом {rank}.")
    setup(rank, world_size)

    # создать модель и отправить её на GPU с id = rank
    model = ToyModel()
    ddp_model = DDP(model)

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    optimizer.zero_grad()
    outputs = ddp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5)
    loss_fn(outputs, labels).backward()
    optimizer.step()

    cleanup()


def run_demo(demo_fn, world_size):
    mp.spawn(demo_fn,
             args=(world_size,),
             nprocs=world_size,
             join=True)
    
def demo_checkpoint(rank, world_size):
    print(f"Пример использованиия DDP с чекпоинтами с рангом {rank}.")
    setup(rank, world_size)

    model = ToyModel().to(rank)
    ddp_model = DDP(model, device_ids=[rank])

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    CHECKPOINT_PATH = tempfile.gettempdir() + "/model.checkpoint"
    if rank == 0:
        # Все процессы должны начать с одних значений параметров и градиента,
        # они также синхронизируются на backward шаге,
        # поэтому достаточно сохранять модель в одном процессе
        torch.save(ddp_model.state_dict(), CHECKPOINT_PATH)

    # Используем barrier(), чтобы удостовериться,
    # что процесс 1 загрузит модель после сохранения процессом 0
    dist.barrier()
    # опциии map_location
    map_location = {'cuda:%d' % 0: 'cuda:%d' % rank}
    ddp_model.load_state_dict(
        torch.load(CHECKPOINT_PATH, map_location=map_location))

    optimizer.zero_grad()
    outputs = ddp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(rank)
    loss_fn = nn.MSELoss()
    loss_fn(outputs, labels).backward()
    optimizer.step()

    if rank == 0:
        os.remove(CHECKPOINT_PATH)

    cleanup()

class ToyMpModel(nn.Module):
    def __init__(self, dev0, dev1):
        super(ToyMpModel, self).__init__()
        self.dev0 = dev0
        self.dev1 = dev1
        self.net1 = torch.nn.Linear(10, 10).to(dev0)
        self.relu = torch.nn.ReLU()
        self.net2 = torch.nn.Linear(10, 5).to(dev1)

    def forward(self, x):
        x = x.to(self.dev0)
        x = self.relu(self.net1(x))
        x = x.to(self.dev1)
        return self.net2(x)

def demo_model_parallel(rank, world_size):
    print(f"Пример использованиия DDP с параллельностью по модели с рангом {rank}.")
    setup(rank, world_size)

    # настроим модель и устройства в этом процессе
    dev0 = rank * 2
    dev1 = rank * 2 + 1
    mp_model = ToyMpModel(dev0, dev1)
    ddp_mp_model = DDP(mp_model)

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_mp_model.parameters(), lr=0.001)

    optimizer.zero_grad()
    # вывод будет на устройстве dev1
    outputs = ddp_mp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(dev1)
    loss_fn(outputs, labels).backward()
    optimizer.step()

    cleanup()

if __name__ == '__main__':
    n_gpus = torch.cuda.device_count()
    print("Количество GPU:", n_gpus)
    run_demo(demo_basic, 4)
    run_demo(demo_checkpoint, n_gpus)
    # run_demo(demo_model_parallel, n_gpus)

Overwriting launch_ddp_demo.py


In [2]:
! python3 launch_ddp_demo.py

Количество GPU: 0
Базовый пример использованиия DDP с рангом 0.
Базовый пример использованиия DDP с рангом 1.
Базовый пример использованиия DDP с рангом 3.
Базовый пример использованиия DDP с рангом 2.


Для сохранениия и загрузки состояния обучения (checkpoint) используются функциии `torch.save()` и `torch.load()`

Model Parallel - использование нескольких устройств на одном обработчике.

### Обучение MNIST

In [5]:
%%writefile launch_ddp.py

import os
from datetime import datetime
import argparse
import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-n', '--nodes', default=1, type=int, metavar='N',
                        help='количество обработчиков (default: 1)')
    parser.add_argument('-nr', '--nr', default=0, type=int,
                        help='глобальный ранг')
    parser.add_argument('--epochs', default=2, type=int, metavar='N',
                        help='количество эпох обучениия')
    args = parser.parse_args()
    args.world_size = args.nodes
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '23334'
    print("Run with args - {}".format(str(args)))
    mp.spawn(train, nprocs=args.nodes, args=(args,))


class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


def train(rank, args):
    dist.init_process_group(backend='gloo', init_method='env://', world_size=args.world_size, rank=rank)
    torch.manual_seed(0)
    
    print("worker {}".format(rank))

    device = torch.device("cpu")

    model = ConvNet()
    torch.cuda.set_device(device)
    model.cuda(device)
    batch_size = 100
    
    # определим loss function и optimizer
    criterion = nn.CrossEntropyLoss().cuda(device)
    optimizer = torch.optim.SGD(model.parameters(), 1e-4)
    # Обернем модель в DDP
    model = nn.parallel.DistributedDataParallel(model, device_ids=[device])
    # Загрузка данных
    train_dataset = torchvision.datasets.MNIST(root='./data',
                                               train=True,
                                               transform=transforms.ToTensor(),
                                               download=True)
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset,
                                                                    num_replicas=args.world_size,
                                                                    rank=rank)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=False,
                                               num_workers=0,
                                               pin_memory=True,
                                               sampler=train_sampler)

    start = datetime.now()
    total_step = len(train_loader)
    for epoch in range(args.epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i + 1) % 100 == 0:
                print('[{}] Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(rank, epoch + 1, args.epochs, i + 1, total_step,
                                                                         loss.item()))
    if rank == 0:
        print("Обучение завершено за: " + str(datetime.now() - start))


if __name__ == '__main__':
    main()

Overwriting launch_ddp.py


In [6]:
! python3 launch_ddp.py --nodes 2

Run with args - Namespace(epochs=2, nodes=2, nr=0, world_size=2)
worker 1
worker 0
Traceback (most recent call last):
  File "launch_ddp.py", line 108, in <module>
    main()
  File "launch_ddp.py", line 27, in main
    mp.spawn(train, nprocs=args.nodes, args=(args,))
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/multiprocessing/spawn.py", line 240, in spawn
    return start_processes(fn, args, nprocs, join, daemon, start_method='spawn')
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/multiprocessing/spawn.py", line 198, in start_processes
    while not context.join():
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/multiprocessing/spawn.py", line 160, in join
    raise ProcessRaisedException(msg, error_index, failed_process.pid)
torch.multiprocessing.spawn.ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/multiprocessing

### Распределенное обучениеи с использованием PyTorch и Horovod для MNIST

In [ ]:
! pip install horovod[pytorch]

In [7]:
%%writefile learn_hvd.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

# Параметры обучения
batch_size = 100
num_epochs = 5
momentum = 0.5
log_interval = 100

def train_one_epoch(model, device, data_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader) * len(data),
                100. * batch_idx / len(data_loader), loss.item()))
            
from time import time
import os

LOG_DIR = os.path.join('./logs/', str(time()), 'MNISTDemo')
os.makedirs(LOG_DIR)

def save_checkpoint(model, optimizer, epoch):
  filepath = LOG_DIR + '/checkpoint-{epoch}.pth.tar'.format(epoch=epoch)
  state = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
  }
  torch.save(state, filepath)

import torch.optim as optim
from torchvision import datasets, transforms

def train(learning_rate):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  train_dataset = datasets.MNIST(
    'data', 
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))
  data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  model = Net().to(device)

  optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

  for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, device, data_loader, optimizer, epoch)
    save_checkpoint(model, optimizer, epoch)

import horovod.torch as hvd

def train_hvd(learning_rate):
  hvd.init()  # Иницииализация
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
  if device.type == 'cuda':
    torch.cuda.set_device(0)

  train_dataset = datasets.MNIST(
    root='data-%d'% hvd.rank(),  # каждый обработчики в своей папке
    train=True, 
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
  )

  from torch.utils.data.distributed import DistributedSampler
  
  train_sampler = DistributedSampler(train_dataset, num_replicas=hvd.size(), rank=hvd.rank())
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)

  model = Net().to(device)
  
  optimizer = optim.SGD(model.parameters(), lr=learning_rate * hvd.size(), momentum=momentum)

  # оборачиваем оптимизатор в Horovod DistributedOptimizer
  optimizer = hvd.DistributedOptimizer(optimizer, named_parameters=model.named_parameters())
  
  # Ставим для всех моделей начальные параметры одинаковыми
  hvd.broadcast_parameters(model.state_dict(), root_rank=0)

  for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, device, train_loader, optimizer, epoch)
    # Сохраняем только в одном обработчике
    if hvd.rank() == 0:
      save_checkpoint(model, optimizer, epoch)
  

if __name__ == '__main__':
  train_hvd(0.001)


Overwriting learn_hvd.py


In [8]:
from learn_hvd import *

### Обучение MNIST

In [9]:
train(learning_rate = 0.001)

/home/ubuntu/learn_hvd.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.327062
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.314169
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.300497
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.284976
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.269949
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.262890
Train Epoch: 2 [0/60000 (0%)]	Loss: 2.243164
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.217157
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.170468
Train Epoch: 2 [30000/60000 (50%)]	Loss: 2.160197
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.101226
Train Epoch: 2 [50000/60000 (83%)]	Loss: 2.046172
Train Epoch: 3 [0/60000 (0%)]	Loss: 1.969879
Train Epoch: 3 [10000/60000 (17%)]	Loss: 1.854494
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.720318
Train Epoch: 3 [30000/60000 (50%)]	Loss: 1.464677
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.502841
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.449594
Train Epoch: 4 [0/60000 (0%)]	Loss: 1.318620
Train Epoch: 4 [10000/60000 (17%)]	Loss: 1.174082
Train Epoch: 4 [2000

### Horovod

In [10]:
! mpirun -np 2 -H localhost:2 python3 learn_hvd.py

Train Epoch: 1 [0/30000 (0%)]	Loss: 2.326104
Train Epoch: 1 [0/30000 (0%)]	Loss: 2.347001
Train Epoch: 1 [10000/30000 (33%)]	Loss: 2.288108
Train Epoch: 1 [10000/30000 (33%)]	Loss: 2.280800
Train Epoch: 1 [20000/30000 (67%)]	Loss: 2.257359
Train Epoch: 1 [20000/30000 (67%)]	Loss: 2.272566
Train Epoch: 2 [0/30000 (0%)]	Loss: 2.265738
Train Epoch: 2 [0/30000 (0%)]	Loss: 2.211262
Train Epoch: 2 [10000/30000 (33%)]	Loss: 2.142014
Train Epoch: 2 [10000/30000 (33%)]	Loss: 2.156749
Train Epoch: 2 [20000/30000 (67%)]	Loss: 2.049343
Train Epoch: 2 [20000/30000 (67%)]	Loss: 2.024050
Train Epoch: 3 [0/30000 (0%)]	Loss: 1.859179
Train Epoch: 3 [0/30000 (0%)]	Loss: 1.907485
Train Epoch: 3 [10000/30000 (33%)]	Loss: 1.710988
Train Epoch: 3 [10000/30000 (33%)]	Loss: 1.564717
Train Epoch: 3 [20000/30000 (67%)]	Loss: 1.435290
Train Epoch: 3 [20000/30000 (67%)]	Loss: 1.591935
Train Epoch: 4 [0/30000 (0%)]	Loss: 1.251025
Train Epoch: 4 [0/30000 (0%)]	Loss: 1.477257
Train Epoch: 4 [10000/30000 (33%)]	Loss: